In [1]:
import os, re
from dotenv import load_dotenv, find_dotenv
from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits import SQLDatabaseToolkit
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import AgentType, create_sql_agent
from langchain import hub
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from sqlalchemy import create_engine, event

In [2]:

# Load .env (GOOGLE_API_KEY, DATABASE_URL, etc.)
load_dotenv(find_dotenv(), override=True)

# Basic sanity checks
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY") or os.getenv("GEMINI_API_KEY")
if not GOOGLE_API_KEY:
    print("⚠️ Set GOOGLE_API_KEY (or GEMINI_API_KEY) in your .env file before running the agent.")

DATABASE_URL = os.getenv("DATABASE_URL")
if not DATABASE_URL:
    print("⚠️ Set DATABASE_URL in your .env file.")

LOG_SQL = (os.getenv("LOG_SQL", "false").lower() == "true")
GEMINI_MODEL = os.getenv("GEMINI_MODEL", "gemini-1.5-flash")

print("Env loaded. Model =", GEMINI_MODEL)

Env loaded. Model = gemini-1.5-flash


In [3]:
# 1) SQLAlchemy engine with timeouts & read-only transactions

engine = create_engine(
    DATABASE_URL,
    pool_pre_ping=True,
    pool_size=5,
    max_overflow=5,
    connect_args={
        # server-side protection against runaway queries (5s)
        "options": "-c statement_timeout=5000 -c idle_in_transaction_session_timeout=5000"
    },
)

@event.listens_for(engine, "connect")
def _enforce_readonly(dbapi_conn, record):
    # Optional belt & suspenders: force read-only transactions
    with dbapi_conn.cursor() as c:
        c.execute("SET default_transaction_read_only = on;")

In [4]:
# 2) LangChain DB wrapper (ALLOWLIST tables for safety)

ALLOWED_TABLES = ["properties", "market_analytics"]
db = SQLDatabase(engine=engine, include_tables=ALLOWED_TABLES)

In [5]:
# 3) Deterministic LLM for reliable SQL generation

llm = ChatGoogleGenerativeAI(
    model=GEMINI_MODEL,  
    temperature=0, # 0 = deterministic
)

# # Quick smoke test (doesn't hit your DB)
# print(llm.invoke("Say 'Gemini SQL agent ready.'").content)

In [6]:
# 4) Toolkit + runtime SQL guardrails (SELECT-only + LIMIT)

toolkit = SQLDatabaseToolkit(db=db, llm=llm)

# Guard: block non-SELECT and inject LIMIT 100 if missing
_original_db_run = db.run
_limit_re = re.compile(r"\bLIMIT\s+\d+", re.IGNORECASE)

def _guarded_run(sql: str, *args, **kwargs):
    up = sql.strip().upper()
    if not up.startswith("SELECT"):
        raise ValueError("Only SELECT queries are permitted.")
    if not _limit_re.search(sql):
        sql += " LIMIT 100"
    return _original_db_run(sql, *args, **kwargs)

db.run = _guarded_run  # enforce guardrails

In [7]:
# Optional: record every executed SQL for debugging (set to True to enable)
LOG_SQL = True
if LOG_SQL:
    def _logging_run(sql: str, *a, **k):
        print("\n--- SQL EXECUTED ---\n", sql)
        return _guarded_run(sql, *a, **k)
    db.run = _logging_run

In [8]:
# 5) Pull LangChain Hub prompt AND inject live schema snippet

# Pull the standard SQL agent system prompt
prompt_template = hub.pull("langchain-ai/sql-agent-system-prompt")

# Fill in dialect + top_k as in your OpenAI version
base_system = prompt_template.format(dialect="PostgreSQL", top_k=5)

# Get a concise schema to reduce hallucinations
schema_snippet = db.get_table_info(ALLOWED_TABLES)

merged_system_prompt = (
    f"{base_system}\n\n"
    "### ADDITIONAL CONTEXT: DATABASE SCHEMA (READ CAREFULLY)\n"
    "Use ONLY these tables/columns. Prefer aggregates. Always include a LIMIT (<=100) unless a single value is asked.\n\n"
    f"{schema_snippet}"
)


In [9]:
# 6) Create the SQL Agent with the merged system prompt

# Build a full chat prompt so we can inject our merged system text + schema
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", merged_system_prompt),
        ("human", "{input}"),
        # Agent needs this scratchpad to reason over intermediate steps
        MessagesPlaceholder("agent_scratchpad"),
    ]
)

agent = create_sql_agent(
    llm,
    db=db,
    prompt=prompt,
    agent_type="tool-calling",  # Gemini supports tool/function calling
    verbose=True,
    top_k=5,                    # default row cap if user doesn't specify
    max_iterations=15,
)

print("Agent ready.")

Agent ready.


In [10]:
# def ask(question: str):
#     """Run a natural-language question through the SQL agent and print the result."""
#     print(f"\n❓ {question}\n" + "-"*60)
#     out = agent.invoke({"input": question})
#     # AgentExecutor may return either a dict or a string depending on version
#     if isinstance(out, dict) and "output" in out:
#         print(out["output"])
#         return out["output"]
#     print(out)
#     return out

def ask(question: str) -> str:
    """Natural language → (SQL) → Answer using merged Hub+Schema prompt."""
    return agent.run(question)

In [11]:
# # Utility: quickly view the schema the agent sees
# print(schema_snippet[:1200] + ("..." if len(schema_snippet) > 1200 else ""))

In [12]:
# 1. Ask about property listings
print(ask("List all active properties in Austin with their price and number of bedrooms."))



> Entering new SQL Agent Executor chain...


C:\Users\acer\AppData\Local\Temp\ipykernel_15272\670101427.py:14: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  return agent.run(question)



Invoking: `sql_db_list_tables` with `{}`


market_analytics, properties
Invoking: `sql_db_schema` with `{'table_names': 'properties'}`



CREATE TABLE properties (
	id UUID DEFAULT gen_random_uuid() NOT NULL, 
	title TEXT NOT NULL, 
	description TEXT, 
	price NUMERIC(12, 2) NOT NULL, 
	address TEXT NOT NULL, 
	city TEXT NOT NULL, 
	state TEXT NOT NULL, 
	zip_code TEXT NOT NULL, 
	property_type TEXT NOT NULL, 
	bedrooms INTEGER, 
	bathrooms NUMERIC(3, 1), 
	square_feet INTEGER, 
	lot_size NUMERIC(10, 2), 
	year_built INTEGER, 
	listing_status TEXT DEFAULT 'active'::text NOT NULL, 
	created_at TIMESTAMP WITH TIME ZONE DEFAULT now(), 
	updated_at TIMESTAMP WITH TIME ZONE DEFAULT now(), 
	CONSTRAINT properties_pkey PRIMARY KEY (id), 
	CONSTRAINT properties_listing_status_check CHECK (listing_status = ANY (ARRAY['active'::text, 'pending'::text, 'sold'::text, 'off_market'::text])), 
	CONSTRAINT properties_property_type_check CHECK (property_type = ANY (ARRAY['house'::text, 'apartment'::text

In [13]:
# 2. Find investment-type property
print(ask("Which property is described as a great rental property? Show its title, price, and year built."))



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


market_analytics, properties
Invoking: `sql_db_schema` with `{'table_names': 'properties'}`



CREATE TABLE properties (
	id UUID DEFAULT gen_random_uuid() NOT NULL, 
	title TEXT NOT NULL, 
	description TEXT, 
	price NUMERIC(12, 2) NOT NULL, 
	address TEXT NOT NULL, 
	city TEXT NOT NULL, 
	state TEXT NOT NULL, 
	zip_code TEXT NOT NULL, 
	property_type TEXT NOT NULL, 
	bedrooms INTEGER, 
	bathrooms NUMERIC(3, 1), 
	square_feet INTEGER, 
	lot_size NUMERIC(10, 2), 
	year_built INTEGER, 
	listing_status TEXT DEFAULT 'active'::text NOT NULL, 
	created_at TIMESTAMP WITH TIME ZONE DEFAULT now(), 
	updated_at TIMESTAMP WITH TIME ZONE DEFAULT now(), 
	CONSTRAINT properties_pkey PRIMARY KEY (id), 
	CONSTRAINT properties_listing_status_check CHECK (listing_status = ANY (ARRAY['active'::text, 'pending'::text, 'sold'::text, 'off_market'::text])), 
	CONSTRAINT properties_property_type_check CHECK (property_type 

In [14]:
# 3. Market analytics summary
print(ask("What was the average and median price in Austin across January to March 2024?"))



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_query` with `{'query': "SELECT AVG(avg_price) as average_price, AVG(median_price) as average_median_price FROM market_analytics WHERE city = 'Austin' AND month_year BETWEEN '2024-01-01' AND '2024-03-01';"}`



--- SQL EXECUTED ---
 SELECT AVG(avg_price) as average_price, AVG(median_price) as average_median_price FROM market_analytics WHERE city = 'Austin' AND month_year BETWEEN '2024-01-01' AND '2024-03-01';
Error: (psycopg2.errors.SyntaxError) syntax error at or near "LIMIT"
LINE 1: ... month_year BETWEEN '2024-01-01' AND '2024-03-01'; LIMIT 100
                                                              ^

[SQL: SELECT AVG(avg_price) as average_price, AVG(median_price) as average_median_price FROM market_analytics WHERE city = 'Austin' AND month_year BETWEEN '2024-01-01' AND '2024-03-01'; LIMIT 100]
(Background on this error at: https://sqlalche.me/e/20/f405)
Invoking: `sql_db_query` with `{'query': "SELECT AVG(avg_pri

In [15]:
# 4. Trend in Austin
print(ask("Describe the market trend in Austin in early 2024."))



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_query` with `{'query': "SELECT market_trend FROM market_analytics WHERE city = 'Austin' AND month_year BETWEEN '2024-01-01' AND '2024-03-31';"}`



--- SQL EXECUTED ---
 SELECT market_trend FROM market_analytics WHERE city = 'Austin' AND month_year BETWEEN '2024-01-01' AND '2024-03-31';
Error: (psycopg2.errors.SyntaxError) syntax error at or near "LIMIT"
LINE 1: ... month_year BETWEEN '2024-01-01' AND '2024-03-31'; LIMIT 100
                                                              ^

[SQL: SELECT market_trend FROM market_analytics WHERE city = 'Austin' AND month_year BETWEEN '2024-01-01' AND '2024-03-31'; LIMIT 100]
(Background on this error at: https://sqlalche.me/e/20/f405)
Invoking: `sql_db_query` with `{'query': "SELECT market_trend FROM market_analytics WHERE city = 'Austin' AND month_year BETWEEN '2024-01-01' AND '2024-03-31' LIMIT 5;"}`



--- SQL EXECUTED ---
 SELECT market_trend FROM market_analytics WHERE ci

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 15
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 44
}
].
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 4.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing

In early 2024, the market trend in Austin was rising.


> Finished chain.
In early 2024, the market trend in Austin was rising.



In [16]:
# 5. Compare cities
print(ask("Compare average property prices between Austin, Dallas, and Houston in January 2024."))



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


market_analytics, properties
Invoking: `sql_db_schema` with `{'table_names': 'market_analytics'}`



CREATE TABLE market_analytics (
	id UUID DEFAULT gen_random_uuid() NOT NULL, 
	city TEXT NOT NULL, 
	state TEXT NOT NULL, 
	avg_price NUMERIC(12, 2), 
	median_price NUMERIC(12, 2), 
	price_per_sqft NUMERIC(8, 2), 
	market_trend TEXT, 
	inventory_level INTEGER, 
	days_on_market INTEGER, 
	month_year DATE NOT NULL, 
	created_at TIMESTAMP WITH TIME ZONE DEFAULT now(), 
	CONSTRAINT market_analytics_pkey PRIMARY KEY (id), 
	CONSTRAINT market_analytics_market_trend_check CHECK (market_trend = ANY (ARRAY['rising'::text, 'stable'::text, 'declining'::text]))
)

/*
3 rows from market_analytics table:
id	city	state	avg_price	median_price	price_per_sqft	market_trend	inventory_level	days_on_market	month_year	created_at
cdeecd57-dc39-4de5-836c-b65d6f1a687b	Austin	TX	425000.00	395000.00	285.50	rising	1250	28	2024-

In [17]:
# 6. Inventory & days on market
print(ask("Show the inventory level and days on market for Dallas in February 2024."))



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_query` with `{'query': "SELECT inventory_level, days_on_market FROM market_analytics WHERE city = 'Dallas' AND month_year = '2024-02-01';"}`



--- SQL EXECUTED ---
 SELECT inventory_level, days_on_market FROM market_analytics WHERE city = 'Dallas' AND month_year = '2024-02-01';
Error: (psycopg2.errors.SyntaxError) syntax error at or near "LIMIT"
LINE 1: ...ERE city = 'Dallas' AND month_year = '2024-02-01'; LIMIT 100
                                                              ^

[SQL: SELECT inventory_level, days_on_market FROM market_analytics WHERE city = 'Dallas' AND month_year = '2024-02-01'; LIMIT 100]
(Background on this error at: https://sqlalche.me/e/20/f405)
Invoking: `sql_db_query` with `{'query': "SELECT inventory_level, days_on_market FROM market_analytics WHERE city = 'Dallas' AND month_year = '2024-02-01' LIMIT 5;"}`



--- SQL EXECUTED ---
 SELECT inventory_level, days_on_market FROM market_analytics WHERE

In [18]:
# 7. High-end listings
print(ask("Which is the most expensive property in the database, and what type is it?"))



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


market_analytics, properties
Invoking: `sql_db_schema` with `{'table_names': 'properties'}`



CREATE TABLE properties (
	id UUID DEFAULT gen_random_uuid() NOT NULL, 
	title TEXT NOT NULL, 
	description TEXT, 
	price NUMERIC(12, 2) NOT NULL, 
	address TEXT NOT NULL, 
	city TEXT NOT NULL, 
	state TEXT NOT NULL, 
	zip_code TEXT NOT NULL, 
	property_type TEXT NOT NULL, 
	bedrooms INTEGER, 
	bathrooms NUMERIC(3, 1), 
	square_feet INTEGER, 
	lot_size NUMERIC(10, 2), 
	year_built INTEGER, 
	listing_status TEXT DEFAULT 'active'::text NOT NULL, 
	created_at TIMESTAMP WITH TIME ZONE DEFAULT now(), 
	updated_at TIMESTAMP WITH TIME ZONE DEFAULT now(), 
	CONSTRAINT properties_pkey PRIMARY KEY (id), 
	CONSTRAINT properties_listing_status_check CHECK (listing_status = ANY (ARRAY['active'::text, 'pending'::text, 'sold'::text, 'off_market'::text])), 
	CONSTRAINT properties_property_type_check CHECK (property_type 